In [1]:
import numpy as np
from numpy import load
import h5py
import pandas as pd
import torch
import lightning as L
from pytorch_lightning import LightningModule
from scipy import linalg
import scipy
import tqdm
from itertools import product

In [2]:
from PL_DeepSTARR import *
from seq_evals_improved import *
from helpers import *

In [3]:
samples_file_path = 'samples.npz'
deepSTARR_data = 'DeepSTARR_data.h5'
oracle_path = 'oracle_DeepSTARR_DeepSTARR_data.ckpt'

In [4]:
deepstarr = load_deepstarr(oracle_path)

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
#load data
x_test_tensor, x_synthetic_tensor = extract_data(samples_file_path, deepSTARR_data)

<h2>Functional similarity: Conditional generation fidelity</h2>

In [15]:
#make predictions
y_hat_test, y_hat_syn = load_predictions(x_test_tensor, x_synthetic_tensor, deepstarr)

In [16]:
#calculate Conditional generation fidelity mse
activity1 = y_hat_syn
activity2 = y_hat_test
mse = conditional_generation_fidelity(activity1, activity2)
print(mse)

1.2618008


<h2>Functional similarity: Frechet distance</h2>

In [6]:
embeddings1 = get_penultimate_embeddings(deepstarr, x_test_tensor)

In [7]:
embeddings2 = get_penultimate_embeddings(deepstarr, x_synthetic_tensor)

In [11]:
mu1, sigma1 = calculate_activation_statistics(embeddings1)

In [12]:
mu2, sigma2 = calculate_activation_statistics(embeddings2)

In [13]:
distance = calculate_frechet_distance(mu1, sigma1, mu2, sigma2)

In [14]:
distance

0.32809665113524034

<h2>Functional similarity: Predictive distribution shift</h2>

In [6]:
predictive_distribution_shift(x_synthetic_tensor, x_test_tensor)

0.007824236293496212

<h2>Sequence similarity: k-mer spectrum shift</h2>

In [6]:
X_test, X_syn = prepare_deepstarr_for_kmer(x_test_tensor, x_synthetic_tensor)

In [7]:
kmer_statistics(3, X_test, X_syn)

100%|██████████████████████████████████| 41186/41186 [00:04<00:00, 10256.48it/s]


(0.001412, 0.018828)

<h2>Sequence similarity: Discriminatability</h2>